In [ ]:
import pygame
from sys import exit
from random import randint

def display_score():
    current_time = int(pygame.time.get_ticks() / 1000)- start_time
    score_surf = test_font.render(f'Score: {current_time}',False,(64,64,64))
    score_rect = score_surf.get_rect(center = (650,25))
    screen.blit(score_surf,score_rect)
    return current_time

def obstacle_movement(obstacle_list):
    if obstacle_list: 
        for obstacle_rect in obstacle_list:
            obstacle_rect.x -= 5
            
            if obstacle_rect.bottom == 500:  
                screen.blit(snail_surface, obstacle_rect)
            else:
                screen.blit(fly_surface, obstacle_rect)
        
        obstacle_list = [obstacle for obstacle in obstacle_list if obstacle.x > 0]
        
        return obstacle_list
    else: 
        return []
    
def collisions(player,obstacles):
    if obstacles:
        for obstacle_rect in obstacles:
            if player.colliderect(obstacle_rect):
                return False
    return True

def player_animation():
    global player_surf, player_index
    
    if player_rect.bottom < 500:
        player_surf = player_jump
    else:
        player_index += 0.1
        if player_index >= len(player_walk):
            player_index = 0
        player_surf = player_walk[int(player_index)]
    

#Game functionality & Globals
pygame.init()
screen = pygame.display.set_mode((800, 600))
pygame.display.set_caption("Karl's game")
clock = pygame.time.Clock()
test_font = pygame.font.Font("Pixeltype.ttf",50)
game_active = True
start_time = 0
score = 0

#Background declared. Convert images to python friendly version.
sky_surface = pygame.image.load('Sky.png').convert()
ground_surface = pygame.image.load("Ground.png").convert()

#Game Name
text_surf = test_font.render("Robyn Runner!", False, (64,64,64))
text_rect = text_surf.get_rect(center = (150,25))

#Enemies & Logic
snail_frame_1 = pygame.image.load("snail1.png").convert_alpha()
snail_frame_2 = pygame.image.load("snail2.png").convert_alpha()
snail_frames = [snail_frame_1,snail_frame_2]
snail_frame_index = 0
snail_surface =snail_frames[snail_frame_index]

fly_frame_1 = pygame.image.load("Fly1.png").convert_alpha()
fly_frame_2 = pygame.image.load("Fly2.png").convert_alpha()
fly_frames = [fly_frame_1,fly_frame_2]
fly_frame_index = 0
fly_surface = fly_frames[fly_frame_index]

#Overall enemy list
obstacle_rect_list = []

#Player declared. #draws a rectangle around the player. #Position, X,Y. Rectangle draw from point. Point in line with Ground Y
player_walk_1 = pygame.image.load("player_walk_1.png").convert_alpha()
player_walk_2 = pygame.image.load("player_walk_2.png").convert_alpha()
player_walk = [player_walk_1, player_walk_2]
player_index = 0
player_jump = pygame.image.load("jump.png").convert_alpha()

player_surf = player_walk[player_index]
player_rect = player_surf.get_rect(midbottom = (100,500))
player_gravity = 0

#Game over interface
player_stand = pygame.image.load("player_stand.png").convert_alpha()
player_stand_scaled = pygame.transform.scale2x(player_stand)
player_stand_rect = player_stand_scaled.get_rect(midbottom = (400,325))

#Game over interface
game_name = test_font.render("Robyn Runner!", False, (111,196,169))
game_name_rect = game_name.get_rect(center = (400,100))

#Game over interface
play_again = test_font.render("Press space to play again!", False, (111,196,169))
play_again_rect = play_again.get_rect(center = (400,400))

#Enemy timers 
obstacle_timer = pygame.USEREVENT + 1
pygame.time.set_timer(obstacle_timer,1100)

snail_animation_timer = pygame.USEREVENT + 2
pygame.time.set_timer(snail_animation_timer,500)

fly_animation_timer = pygame.USEREVENT + 3
pygame.time.set_timer(fly_animation_timer,200)

while True: 
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            exit()      
            
        if game_active == False and event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
            game_active = True
        
        if game_active:
            if event.type == obstacle_timer:
                    if randint(0,2):
                        obstacle_rect_list.append(snail_surface.get_rect(midbottom = (randint(900,1100),500)))
                    else:
                        obstacle_rect_list.append(fly_surface.get_rect(midbottom = (randint(900,1100),350)))
            if event.type == snail_animation_timer:
                if snail_frame_index == 0:
                    snail_frame_index = 1
                else:
                    snail_frame_index = 0
                snail_surface = snail_frames[snail_frame_index]
                
            if event.type == fly_animation_timer:
                if fly_frame_index == 0:
                    fly_frame_index = 1
                else:
                    fly_frame_index = 0
                fly_surface = fly_frames[fly_frame_index]
        
    if game_active:
        #Background shown
        screen.blit(sky_surface,(0,0))
        screen.blit(ground_surface,(0,500))
        screen.blit(text_surf,text_rect)
        score = display_score()

        #Enemy Movement
        obstacle_rect_list = obstacle_movement(obstacle_rect_list)
        game_active = collisions(player_rect,obstacle_rect_list)
    
        #Player object, Floor & Jump Height
        player_gravity +=1
        player_rect.y += player_gravity
        player_animation()
        if player_rect.bottom >= 500:
            player_rect.bottom = 500

        screen.blit(player_surf,player_rect)
        #Jump input    
        keys = pygame.key.get_pressed()
        if keys[pygame.K_SPACE] and player_rect.bottom == 500:
            player_gravity = -20
    else:
        #Make screen this colour, show various objects for score, instructions, player object for QOL 
        #Resets player and objects for next game if needed.
        screen.fill((94,129,162))
        screen.blit(player_stand_scaled, player_stand_rect) 
        obstacle_rect_list.clear()
        player_rect.midbottom = (100,500)
        player_gravity = 0
        
        score_message = test_font.render(f'Your score: {score}',False,(111,196,169))
        score_message_rect = score_message.get_rect(center = (400, 350))
        screen.blit(game_name, game_name_rect)
            
        if score == 0:
            screen.blit(play_again, play_again_rect)
        else:
            screen.blit(play_again, play_again_rect)
            screen.blit(score_message, score_message_rect)
            start_time = int(pygame.time.get_ticks() / 1000)
            
    pygame.display.update()
    clock.tick(60)

pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html
